In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['embeddings', 'train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
import pandas as pd

import gensim
from gensim.models.doc2vec import TaggedDocument

In [3]:
train_csv = "../input/train.csv"
test_csv = "../input/test.csv"
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)
train_df.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [4]:
test_df.head()

,qid,question_text
0,00014894849d00ba98a9,My voice range is A2-C5. My chest voice goes u...
1,000156468431f09b3cae,How much does a tutor earn in Bangalore?
2,000227734433360e1aae,What are the best made pocket knives under $20...
3,0005e06fbe3045bd2a92,Why would they add a hypothetical scenario tha...
4,00068a0f7f41f50fc399,What is the dresscode for Techmahindra freshers?


In [5]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize, TweetTokenizer
import re

In [6]:
def remove_stopwords(words):
    stop_words = set(stopwords.words("english"))
    return [word for word in words if word not in stop_words]

def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

def lemmatize_text(words):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def stem_text(words):
    ps = PorterStemmer()
    return [ps.stem(word) for word in words]

In [7]:
%%time
# Removing punctuation
train_df["question_text"] = train_df["question_text"].apply(lambda x: remove_punctuation(x))
test_df["question_text"] = test_df["question_text"].apply(lambda x: remove_punctuation(x))

# Tokenizing the text
train_df["question_text"] = train_df["question_text"].apply(lambda x: word_tokenize(x))
test_df["question_text"] = test_df["question_text"].apply(lambda x: word_tokenize(x))

# Removing stopwords
# train_df["question_text"] = train_df["question_text"].apply(lambda x: remove_stopwords(x))
# test_df["question_text"] = test_df["question_text"].apply(lambda x: remove_stopwords(x))

# Lemmatizting
train_df["question_text"] = train_df["question_text"].apply(lambda x: lemmatize_text(x))
test_df["question_text"] = test_df["question_text"].apply(lambda x: lemmatize_text(x))

CPU times: user 4min, sys: 1.88 s, total: 4min 2s
Wall time: 4min 2s


In [8]:
train_df.head()

,qid,question_text,target
0,00002165364db923c7e6,"[How, did, Quebec, nationalist, see, their, pr...",0
1,000032939017120e6e44,"[Do, you, have, an, adopted, dog, how, would, ...",0
2,0000412ca6e4628ce2cf,"[Why, doe, velocity, affect, time, Does, veloc...",0
3,000042bf85aa498cd78e,"[How, did, Otto, von, Guericke, used, the, Mag...",0
4,0000455dfa3e01eae3af,"[Can, I, convert, montra, helicon, D, to, a, m...",0


In [9]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

In [ ]:
%%time
train_df["vector"] = train_df["question_text"].apply(lambda x:
                    Doc2Vec(document=x,
                            min_count=1,
                             iter=55,
                            vector_size=150))

test_df["vector"] = train_df["question_text"].apply(lambda x:
                    Doc2Vec(document=x,
                            min_count=1,
                            iter=55,
                            vector_size=150))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
"""
%%time
vectorizer = TfidfVectorizer(use_idf=True, binary=True)
train_df["vector"] = train_df["question_text"].apply(lambda x: vectorizer.fit_transform(x))
test_df["vector"] = test_df["question_text"].apply(lambda x: vectorizer.fit_transform(x))
"""

# ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
train_df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    train_df["vector"], train_df["target"], test_size=0.3)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

In [ ]:
print(lr.score(X_test, y_test))

In [ ]:
lr.fit(train_df["vector"], train_df["target"])
y_predict = lr.predict(test_df["vector"])
predict = pd.Dataframe(data=y_predict, columns=["predictiion"])
predict = predict.astype(int)

id = test_df["qid"]
id_df = pd.DataFrame(id)

result = id_df.join(predict)
result.to_csv("submission.csv", index=False)